unsupervised density-based clustering to build a visual map of word clouds.

maili gruplar halinde bölebilir mi
sıraya koyabilir mi
fikir: benim istemediğim mailleri doğrudan başka bir folder'a yollasın. ben işaretledikçe 

In [ ]:
import pickle

from exchangelib import Credentials, Account,Message
from dotenv import load_dotenv
import os 


mail = os.environ.get('secretUser')
password = os.environ.get('secretKey')
print(mail)
credentials = Credentials(mail, password)
account = Account(mail, credentials=credentials, autodiscover=True)



In [ ]:

import os, re, os.path
count = 0
cwd = os.path.join(os.getcwd() , "mailbox")
for root, dirs, files in os.walk(cwd):
    for file in files:
        os.remove(os.path.join(root, file))
        count = count +  1

print(" {0} files deleted".format(count))

    #for item in account.inbox.all():
#for item in account.inbox.all().order_by('-datetime_received'):
count = 0
for _i in range(0,account.inbox.total_count,100):
    for item in account.inbox.all().order_by('-datetime_received')[_i:_i+100]:
        #
        print(count,item.subject, item.sender, item.datetime_received,item.changekey)    
        #save it
        path = os.path.join(cwd,'{0}.pickle'.format(item.changekey.replace('/','_')))
        with open(path, 'wb') as file:
            pickle.dump(item, file) 
        count = count + 1
print("{0} items created.".format(count))


In [ ]:
import os
folder = os.getcwd() + '/mailbox/'
files = os.listdir(folder)


count =  0
appended_list = []
for file in files:
    #print(file)
    item = Message()
    with open(folder + file, 'rb') as file:
        item = pickle.load(file) 
    #print(item.subject)
    #print(item.sender)
    #print(item.datetime_received)
    appended_list.append({"subject":item.subject,"sender":item.sender.name,"datetime_received": item.datetime_received})
    #print("to_recipients::",item.to_recipients)
    #print(item.received_by)
    #if count == 5:
    #    break
    #count = count  + 1


In [ ]:
import pandas as pd
df  = pd.DataFrame( appended_list, columns=[ 'subject', 'sender', 'datetime_received'])
df


In [ ]:
df_filtered = df[df['datetime_received'].dt.strftime('%Y-%m') == '2022-07']
df_filtered

In [ ]:
df_filtered[df_filtered['subject']==None]

In [ ]:
#Codes by Ragnar https://www.kaggle.com/rowhitswami/starter-load-stopwords

def get_stopwords_list(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return list(frozenset(stop_set))

In [ ]:
folder = os.getcwd() + '/data/'

stopwords_path = folder+"turkish.txt"
stopwords = get_stopwords_list(stopwords_path)


In [ ]:
# -*- coding: utf8 -*-
# Redesigned by @guneysus

lcase_table = tuple(u'abcçdefgğhıijklmnoöprsştuüvyz')
ucase_table = tuple(u'ABCÇDEFGĞHIİJKLMNOÖPRSŞTUÜVYZ')

def upper(data):
    data = data.replace('i',u'İ')
    data = data.replace(u'ı',u'I')
    result = ''
    for char in data:
        try:
            char_index = lcase_table.index(char)
            ucase_char = ucase_table[char_index]
        except:
            ucase_char = char
        result += ucase_char
    return result

def lower(data):
    data = data.replace(u'İ',u'i')
    data = data.replace(u'I',u'ı')
    result = ''
    for char in data:
        try:
            char_index = ucase_table.index(char)
            lcase_char = lcase_table[char_index]
        except:
            lcase_char = char
        result += lcase_char
    return result

def capitalize(data):
    return data[0].upper() + data[1:].lower()

def title(data):
    return " ".join(map(lambda x: x.capitalize(), data.split()))


print (u'istanbul'.upper())
print (u'İSTANBUL'.lower())
print (lower('İSTANBUL'))


In [ ]:
stopwords.append("re:")
stopwords.append("fw:")
stopwords.append("hk")
stopwords.append("hk.")
stopwords.append("hakkında")


In [ ]:
def remove_stop_words(corpus):
    results = []
    for text in corpus:
        if text == None:
            continue
        text =  text.lower()
        tmp = text.split(' ')
        for stop_word in stopwords:
            if stop_word in tmp:
                tmp.remove(stop_word)
        results.append(" ".join(tmp))
        
    return results

In [ ]:
corpus =  df['subject'].tolist()
corpus = remove_stop_words(corpus)


In [ ]:
single_corpus = ""
for c in corpus:
   single_corpus = single_corpus +" " + c 


In [ ]:
from wordcloud import WordCloud
wordcloud = WordCloud().generate(single_corpus)


In [ ]:
# Display the generated image:
# the matplotlib way:
import matplotlib.pyplot as plt
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")

# lower max_font_size
wordcloud = WordCloud(max_font_size=60).generate(single_corpus)
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
# filter by single day
df_filtered = df[df['date'].dt.strftime('%Y-%m-%d') == '2014-01-01']

# filter by single month
df_filtered = df[df['date'].dt.strftime('%Y-%m') == '2014-01']

# filter by single year
df_filtered = df[df['date'].dt.strftime('%Y') == '2014']

In [ ]:
from datetime import date

import pandas as pd

value_to_check = pd.Timestamp(date.today().year, 1, 1)
filter_mask = df['date_column'] < value_to_check
filtered_df = df[filter_mask]

from datetime import datetime
from dateutil.relativedelta import relativedelta
given_date = '21/1/2021'
print('Give Date: ', given_date)
date_format = '%d/%m/%Y'
dtObj = datetime.strptime(given_date, date_format)
# Subtract 20 months from a given datetime object
n = 20
past_date = dtObj - relativedelta(months=n)
print('Past Date: ', past_date)
print('Past Date: ', past_date.date())
# Convert datetime object to string in required format
past_date_str = past_date.strftime(date_format)
print('Past Date as string object: ', past_date_str)

import pickle
#save it
with open(f'test.pickle', 'wb') as file:
    pickle.dump(item, file) 

#load it
with open(f'test.pickle', 'rb') as file2:
    s1_new = pickle.load(file2)
s1_new